In [27]:
import pandas as pd 


sp400 = pd.read_csv('index/sp400_mid_cap.csv')
sp400.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location
0,AA,Alcoa,Materials,Aluminum,"Pittsburgh, Pennsylvania"
1,AAL,American Airlines Group,Industrials,Passenger Airlines,"Fort Worth, Texas"
2,AAON,AAON,Industrials,Building Products,"Tulsa, Oklahoma"
3,ACHC,Acadia Healthcare,Health Care,Health Care Facilities,"Franklin, Tennessee"
4,ACI,Albertsons,Consumer Staples,Food Retail,"Boise, Idaho"


In [30]:
assert sp400['Symbol'].notna().all()

In [31]:
symbols = sp400['Symbol'].to_list()

In [32]:
from src.utils import download_max

df = download_max(symbols=symbols)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  401 of 401 completed
/home/ping/Code/proto-collect-stock-data/src/utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
df.tail()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
2716926,2025-05-30,CMC,46.00,46.84,45.50,46.59,1362200
2716927,2025-05-30,RYN,23.68,23.98,23.60,23.70,1551800
2716928,2025-05-30,BRX,25.62,25.74,25.24,25.41,2572300
2716929,2025-05-30,MMS,71.99,72.82,71.62,72.51,578100
2716930,2025-05-30,LPX,90.06,90.49,89.52,90.07,728700


as there already stock data of sp500 companies in the stock_prices table, it may be better to trim the dataframe here to match the most recent date of the existing data in stock_prices table, as this will make future data fetching easier.

In [38]:
from src.utils import get_db_engine

engine = get_db_engine()

In [40]:
from importlib import reload
from src import utils
reload(utils)

from src.utils import get_latest_record_date

latest_df = get_latest_record_date(engine=engine)
latest_df.head()

,ticker,latest_record_date
0,A,2025-05-08
1,AAPL,2025-05-08
2,ABBV,2025-05-08
3,ABNB,2025-05-08
4,ABT,2025-05-08


check if all tickers share the same latest record date

In [41]:
assert (latest_df['latest_record_date'] == latest_df['latest_record_date'].max()).all()

In [46]:
date_latest_on_db = latest_df['latest_record_date'].max()
date_latest_on_db

datetime.date(2025, 5, 8)

In [47]:
df_trimmed = df[df['trade_date'] <= date_latest_on_db]
df_trimmed.tail()

,trade_date,ticker,open_price,high_price,low_price,close_price,volume
2710911,2025-05-08,CMC,45.48,46.61,45.20,46.15,724000
2710912,2025-05-08,RYN,23.61,23.79,23.33,23.37,1198900
2710913,2025-05-08,BRX,25.36,25.50,24.87,25.11,2138100
2710914,2025-05-08,MMS,70.26,76.07,69.08,75.44,1188700
2710915,2025-05-08,LPX,90.74,91.42,89.56,90.09,544500


In [48]:
data = df_trimmed.to_dict(orient='records')

In [49]:
from src.utils import insert_data
from src.create_sql_tables import StockPrice

insert_data(data=data, table=StockPrice)